In [26]:
import insightface
import numpy as np
import os
import cv2
import glob
from insightface.app.common import Face
from insightface.model_zoo import model_zoo
from insightface.model_zoo import model_zoo
from tqdm import tqdm



In [27]:
det_model_path = 'buffalo_s/det_500m.onnx'
rec_model_path = 'buffalo_s/w600k_mbf.onnx'

det_model = model_zoo.get_model(f'/Users/tayamayuto/Face_recognition_IF/.insightface/models/{det_model_path}')
rec_model = model_zoo.get_model(f'/Users/tayamayuto/Face_recognition_IF/.insightface/models/{rec_model_path}')

det_model.prepare(ctx_id=0, input_size=(640, 640), det_thres=0.5)

Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}
Applied providers: ['CPUExecutionProvider'], with options: {'CPUExecutionProvider': {}}


/Users/tayamayuto/Face_recognition_IF/face_env/lib/python3.10/site-packages/onnxruntime/capi/onnxruntime_inference_collection.py:65: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn(


In [28]:
known_names = []
unknown_names = []
known_embeddings = []
unknown_embeddings = []

players = os.listdir('/Users/tayamayuto/Face_recognition_IF/data/raw')

for player in tqdm(players):
    player_embeddings, player_names = [], []

    img_paths = glob(f'/Users/tayamayuto/Face_recognition_IF/data/raw/{player}/*')
    for img_path in img_paths:
        img = cv2.imread(img_path)
        if img is None: continue

        bboxes, kpss = det_model.detect(img, max_num=0, metric='defualt')
        if len(bboxes) != 1: continue

        bbox = bboxes[0, :4]
        det_score = bboxes[0, 4]
        kps = kpss[0]
        face = Face(bbox=bbox, kps=kps, det_score=det_score)

        rec_model.get(img, face)
        player_embeddings.append(face.normed_embedding)
        player_names.append(player)
        if len(player_embeddings) == 10: break
    
    player_embeddings = np.stack(player_embeddings, axis=0)
    known_embeddings.append(player_embeddings[0:5])
    unknown_embeddings.append(player_embeddings[5:10])
    known_names += player_names[0:5]
    unknown_names += player_names[5:10]

known_embeddings = np.concatenate(known_embeddings, axis=0)
unknown_embeddings = np.concatenate(unknown_embeddings, axis=0)


FileNotFoundError: [Errno 2] No such file or directory: '/Users/tayamayuto/Face_recognition_IF/data/raw'

In [ ]:
def get_averages(names, scores):
    d = defaultdict(list)
    for n, s in zip(names, scores):
        d[n].append(s)

    averages = {}
    for n, s in d.items():
        averages[n] = np.mean(s)
    
    return averages

In [ ]:
def search_average(known_embeddings, known_names, unknown_embeddings, threshold=0.5):
    pred_names = []
    for emb in unknown_embeddings:
        scores = np.dot(emb, known_embeddings.T)
        scores = np.clip(scores, 0., 1.)

        averages = get_averages(known_names, scores)
        pred = sorted(averages, key=lambda x: averages[x], reverse=True)[0]
        score = averages[pred]

        if score > threshold:
            pred_names.append(pred)
        else:
            pred_names.append(None)
    
    return pred_names

In [ ]:
def evaluate(true_names, pred_names):
    coverage = np.mean([n is not None for n in pred_names]) * 100.

    is_corrects = []
    for t, p in zip(true_names, pred_names):
        if p is None: continue
        is_corrects.append(t == p)
    
    if not is_corrects:
        is_corrects.append(False)

    accuracy = np.mean(is_corrects) * 100.
    return accuracy, coverage